# 04 Noise Generation

In this notebook, we talk about how to generate noise. We will cover multiple types of noise as they are added.

In [ ]:
import os
from matplotlib import pyplot as plt

from ananke.configurations.presets.detector import single_line_configuration
from ananke.schemas.event import NoiseType
from ananke.services.detector import DetectorBuilderService
from ananke.visualisation.event import draw_hit_histogram, draw_hit_distribution
from olympus.configuration.generators import NoiseGeneratorConfiguration, BioluminescenceGeneratorConfiguration
from ananke.configurations.collection import HDF5StorageConfiguration
from olympus.event_generation.generators import generate
from olympus.configuration.generators import (
    DatasetConfiguration,
    GenerationConfiguration,
)

Let's get our detector

In [ ]:
detector_service = DetectorBuilderService()
det = detector_service.get(configuration=single_line_configuration)

## Electrical Noise Generation

In [ ]:
electrical_data_path='data/electrical_noise.h5'
noise_generator_config = NoiseGeneratorConfiguration(
    type=NoiseType.ELECTRICAL,
    start_time=0,
    duration=1000,
)

dataset_configuration = DatasetConfiguration(
    detector=single_line_configuration,
    generators=[
        GenerationConfiguration(
            generator=noise_generator_config,
            number_of_samples=2000
        ),
    ],
    storage=HDF5StorageConfiguration(data_path=electrical_data_path, read_only=False)
)

try:
    os.remove(electrical_data_path)
except OSError:
    pass

electrical_collection = generate(dataset_configuration)

In [ ]:
with electrical_collection:
    records = electrical_collection.storage.get_records()
    hits = electrical_collection.storage.get_hits()
    first_record_hits = electrical_collection.storage.get_hits(record_ids=0)
    second_record_hits = electrical_collection.storage.get_hits(record_ids=1)
    detector = electrical_collection.storage.get_detector()
records.df.head()
hits.df

In [ ]:
fig = draw_hit_distribution(first_record_hits)
fig.savefig('data/electrical_first_record_hist.png', dpi=300)
fig.show()

In [ ]:
fig = draw_hit_distribution(second_record_hits)
fig.savefig('data/electrical_second_record_hist.png', dpi=300)
fig.show()

In [ ]:
fig = draw_hit_distribution(hits)
fig.savefig('data/electrical_all_records_hist.png', dpi=300)
fig.show()

In [ ]:
fig = draw_hit_histogram(first_record_hits, detector=detector, colorbar_step=1)
fig.savefig('data/electrical_first_record_pmt.png', dpi=300)
plt.show()

In [ ]:
fig = draw_hit_histogram(second_record_hits, detector=detector)
fig.savefig('data/electrical_second_record_pmt.png', dpi=300)
plt.show()

In [ ]:
fig = draw_hit_histogram(hits, detector=detector)
fig.savefig('data/electrical_all_records_pmt.png', dpi=300)
plt.show()

## Bioluminescense Generation

In [ ]:
bioluminescense_data_path='data/bioluminescense.h5'

bioluminescense_generator_config = BioluminescenceGeneratorConfiguration(
    type=NoiseType.BIOLUMINESCENCE,
    start_time=0,
    duration=1000,
    julia_data_path='../../data/biolumi_sims',
    batch_size= 48
)

dataset_configuration = DatasetConfiguration(
    detector=single_line_configuration,
    generators=[
        GenerationConfiguration(
            generator=bioluminescense_generator_config,
            number_of_samples=2000
        ),
    ],
    storage=HDF5StorageConfiguration(data_path=bioluminescense_data_path, read_only=False)
)


try:
    os.remove(bioluminescense_data_path)
except OSError:
    pass

bioluminescense_collection = generate(dataset_configuration)

In [ ]:
with bioluminescense_collection:
    records = bioluminescense_collection.storage.get_records()
    hits = bioluminescense_collection.storage.get_hits()
    first_record_hits = bioluminescense_collection.storage.get_hits(record_ids=10)
    second_record_hits = bioluminescense_collection.storage.get_hits(record_ids=1)
    detector = bioluminescense_collection.storage.get_detector()
records.df

In [ ]:
fig = draw_hit_distribution(first_record_hits)
fig.savefig('data/bioluminescense_first_record_hist.png', dpi=300)
fig.show()

In [ ]:
fig = draw_bars(second_record_hits.df)
fig.savefig('data/bioluminescense_second_record_hist.png', dpi=300)
fig.show()

In [ ]:
fig = draw_bars(hits.df)
fig.savefig('data/bioluminescense_all_records_hist.png', dpi=300)
fig.show()

In [ ]:
fig = draw_histogram(first_record_hits.df, colorbar_step=1)
fig.savefig('data/bioluminescense_first_record_pmt.png', dpi=300)
plt.show()

In [ ]:
fig = draw_histogram(second_record_hits.df)
fig.savefig('data/bioluminescense_second_record_pmt.png', dpi=300)
plt.show()

In [ ]:
fig = draw_histogram(hits.df, colorbar_step=20)
fig.savefig('data/bioluminescence_all_records_pmt.png', dpi=300)
plt.show()